In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import numpy as np
import warnings
import pandas as pd
import rdkit, rdkit.Chem, rdkit.Chem.rdDepictor, rdkit.Chem.Draw
import networkx as nx
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.utils.data as data
import torch.optim as optim
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import dense_to_sparse, add_self_loops, to_scipy_sparse_matrix
from torch_geometric.data import Data
import torch.nn.functional as F
from six.moves import urllib
import deepchem as dc
import random

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
np.random.seed(0)
random.seed(0)

warnings.filterwarnings("ignore")
sns.set_context("notebook")
sns.set_style(
    "dark",
    {
        "xtick.bottom": True,
        "ytick.left": True,
        "xtick.color": "#666666",
        "ytick.color": "#666666",
        "axes.edgecolor": "#666666",
        "axes.linewidth": 0.8,
        "figure.dpi": 300,
    },
)
color_cycle = ["#1BBC9B", "#F06060", "#5C4B51", "#F3B562", "#6e5687"]
mpl.rcParams["axes.prop_cycle"] = mpl.cycler(color=color_cycle)

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

soldata = pd.read_csv('https://dataverse.harvard.edu/api/access/datafile/3407241?format=original&gbrecs=true')
# had to rehost because dataverse isn't reliable
soldata = pd.read_csv(
    "https://github.com/whitead/dmol-book/raw/main/data/curated-solubility-dataset.csv"
)

/Users/adityabehal/opt/anaconda3/envs/custom-gnn/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
def gen_smiles2graph(smiles):
    """Argument for the RD2NX function should be a valid SMILES sequence
    returns: the graph
    """
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
    out = featurizer.featurize(smiles)
    return out[0]

In [3]:
dc.feat.MolGraphConvFeaturizer?

In [4]:
testCO = gen_smiles2graph("CO")
print(testCO)
print(type(testCO))
print(testCO.node_features)
print(type(testCO.node_features))
print(testCO.edge_index)
print(type(testCO.edge_index))
print(testCO.edge_features)
print(type(testCO.edge_features))

GraphData(node_features=[2, 30], edge_index=[2, 2], edge_features=[2, 11])
<class 'deepchem.feat.graph_data.GraphData'>
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>
[[0 1]
 [1 0]]
<class 'numpy.ndarray'>
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>


In [5]:
help(testCO)

Help on GraphData in module deepchem.feat.graph_data object:

class GraphData(builtins.object)
 |  GraphData(node_features: numpy.ndarray, edge_index: numpy.ndarray, edge_features: Union[numpy.ndarray, NoneType] = None, node_pos_features: Union[numpy.ndarray, NoneType] = None, **kwargs)
 |  
 |  GraphData class
 |  
 |  This data class is almost same as `torch_geometric.data.Data
 |  <https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data>`_.
 |  
 |  Attributes
 |  ----------
 |  node_features: np.ndarray
 |    Node feature matrix with shape [num_nodes, num_node_features]
 |  edge_index: np.ndarray, dtype int
 |    Graph connectivity in COO format with shape [2, num_edges]
 |  edge_features: np.ndarray, optional (default None)
 |    Edge feature matrix with shape [num_edges, num_edge_features]
 |  node_pos_features: np.ndarray, optional (default None)
 |    Node position matrix with shape [num_nodes, num_dimensions].
 |  num_nodes: int
 |    The

In [6]:
testCO_pyg_graph = testCO.to_pyg_graph()

print(testCO_pyg_graph)
print(testCO_pyg_graph.num_nodes)
print(testCO_pyg_graph.is_directed())
print(testCO_pyg_graph.num_node_features)
print(testCO_pyg_graph.num_edge_features)
print(testCO_pyg_graph.keys)

Data(x=[2, 30], edge_index=[2, 2], edge_attr=[2, 11])
2
False
30
11
['x', 'edge_attr', 'edge_index']


In [7]:
testOCO = gen_smiles2graph("OCO")
print(testOCO)
print(type(testOCO))
print(testOCO.node_features)
print(type(testOCO.node_features))
print(testOCO.edge_index)
print(type(testOCO.edge_index))
print(testOCO.edge_features)
print(type(testOCO.edge_features))

GraphData(node_features=[3, 30], edge_index=[2, 4], edge_features=[4, 11])
<class 'deepchem.feat.graph_data.GraphData'>
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 1. 0. 0. 0.]]
<class 'numpy.ndarray'>
[[0 2 2 1]
 [2 0 1 2]]
<class 'numpy.ndarray'>
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>


In [8]:
help(testOCO)

Help on GraphData in module deepchem.feat.graph_data object:

class GraphData(builtins.object)
 |  GraphData(node_features: numpy.ndarray, edge_index: numpy.ndarray, edge_features: Union[numpy.ndarray, NoneType] = None, node_pos_features: Union[numpy.ndarray, NoneType] = None, **kwargs)
 |  
 |  GraphData class
 |  
 |  This data class is almost same as `torch_geometric.data.Data
 |  <https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data>`_.
 |  
 |  Attributes
 |  ----------
 |  node_features: np.ndarray
 |    Node feature matrix with shape [num_nodes, num_node_features]
 |  edge_index: np.ndarray, dtype int
 |    Graph connectivity in COO format with shape [2, num_edges]
 |  edge_features: np.ndarray, optional (default None)
 |    Edge feature matrix with shape [num_edges, num_edge_features]
 |  node_pos_features: np.ndarray, optional (default None)
 |    Node position matrix with shape [num_nodes, num_dimensions].
 |  num_nodes: int
 |    The

In [9]:
# look up smiles strings of chemical structures: https://cactus.nci.nih.gov/chemical/structure
testSO4 = gen_smiles2graph("O[S](O)(=O)=O")
print(testSO4)
print(type(testSO4))
print(testSO4.node_features)
print(type(testSO4.node_features))
print(testSO4.edge_index)
print(type(testSO4.edge_index))
print(testSO4.edge_features)
print(type(testSO4.edge_features))

GraphData(node_features=[5, 30], edge_index=[2, 8], edge_features=[8, 11])
<class 'deepchem.feat.graph_data.GraphData'>
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  1. 0. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>
[[3 4 4 0 4 2 4 1]
 [4 3 0 4 2 4 1 4]]
<class 'numpy.ndarray'>
[[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>


In [10]:
# look up smiles strings of chemical structures: https://cactus.nci.nih.gov/chemical/structure
# node features: ["C", "N", "O", "F", "P", "S", "Cl", "Br", "I"]
testHCN = gen_smiles2graph("[O-][O+]=O")
print(testHCN)
print(type(testHCN))
print(testHCN.node_features)
print(type(testHCN.node_features))
print(testHCN.edge_index)
print(type(testHCN.edge_index))
print(testHCN.edge_features)
print(type(testHCN.edge_features))

GraphData(node_features=[3, 30], edge_index=[2, 4], edge_features=[4, 11])
<class 'deepchem.feat.graph_data.GraphData'>
[[ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.
   0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.
   1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0. -1.  0.  1.  0.  0.  0.  0.  0.
   1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]
<class 'numpy.ndarray'>
[[2 0 0 1]
 [0 2 1 0]]
<class 'numpy.ndarray'>
[[1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0.]]
<class 'numpy.ndarray'>


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [12]:
graph = []
sol = []
for i in range(len(soldata)):
    graph.append(gen_smiles2graph(soldata.SMILES[i]))
    sol.append(soldata.Solubility[i])

Failed to featurize datapoint 0, [Mo]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Al+3].[Al+3].[Mo].[Mo].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Ca+2].[Mg+2].[O-2].[O-2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Mg+2]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Ca+2].[OH-].[OH-]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Ba+2].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[Fe+3].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2]. Appending em

Exception message: tuple index out of range
[15:51:09] WARNING: not removing hydrogen atom without neighbors
[15:51:09] WARNING: not removing hydrogen atom without neighbors
Failed to featurize datapoint 0, [H-].[H-].[Zr+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Mg+2].[Mg+2].[Nb+5].[Nb+5].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2].[O-2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Hf]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [Hf+4].[O-2].[O-2]. Appending empty array
Exception message: tuple index out of range
[15:51:10] WARNING: not removing hydrogen atom without neighbors
[15:51:11] WARNING: not removing hydrogen atom without neighbors
[15:51:11] WARNING: not removing hydrogen atom without neighbors
Failed to featurize datapoint 0, [Cl-].[Li+]. Appending empty a

Exception message: tuple index out of range
Failed to featurize datapoint 0, S.[Ni]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [S-2].[S-2].[Sn+4]. Appending empty array
Exception message: tuple index out of range
[15:51:21] WARNING: not removing hydrogen atom without neighbors
[15:51:21] WARNING: not removing hydrogen atom without neighbors
Failed to featurize datapoint 0, [Cl-].[Cu+2].[Cu+2].[OH-].[OH-].[OH-]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Mn]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featurize datapoint 0, [F-].[F-].[F-].[La+3]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 0, [Ta]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
Failed to featur

In [13]:
class CustomDataset(data.Dataset):
    def __init__(self, graphAll, solAll, transform=None, target_transform=None):
        self.graphInstances = graphAll
        self.solInstances = solAll
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.graphInstances)

    def __getitem__(self, idx):
        graphInstance = self.graphInstances[idx]
        solInstance = self.solInstances[idx]
        if self.transform:
            graphInstance = self.transform(graphInstance)
        if self.target_transform:
            solInstance = self.target_transform(solInstance)
        return graphInstance, solInstance

In [14]:
import multiprocessing

cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

16

In [15]:
dataset = CustomDataset(graphAll=graph, solAll=sol)
dataloader = data.DataLoader(dataset, batch_size=1,
                        shuffle=True, num_workers=cores)
# print(len(dataloader))
# print(type(dataloader))
test_data, val_data, train_data = data.random_split(dataloader, [200, 200, len(dataloader) - 400], generator=torch.Generator().manual_seed(0))

test_data_loader = data.DataLoader(test_data, batch_size=1, shuffle=True, num_workers=cores)
val_data_loader = data.DataLoader(val_data, batch_size=1, shuffle=True, num_workers=cores)
train_data_loader = data.DataLoader(train_data, batch_size=1, shuffle=True, num_workers=cores)
# print(test_data)
# print(val_data)
# print(train_data)
# print(train_data.__getitem__(0))

In [133]:
# create "zeroth" FCNN with 1 fully connected layer
# condense node features from 30 to 24
# dilate edge features from 11 to 24
# see Table S4 BDNCM input feature embedding size 24: https://www.rsc.org/suppdata/d0/sc/d0sc05251e/d0sc05251e1.pdf
class InitialEmbedding(torch.nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        torch.manual_seed(0)
        self.fc_initial_embedding = nn.Linear(c_in, c_out)
    
    def forward(self, features):
        features = self.fc(features)
        features = F.relu(features)
        
        return features

# neural network with two fully connected layers
class FCNN(torch.nn.Module):
    def __init__(self, c_in1, c_out1, c_out2):
        super().__init__()
        torch.manual_seed(0)
        self.fc1 = nn.Linear(c_in1, c_out1)
        torch.manual_seed(0)
        self.fc2 = nn.Linear(c_out1, c_out2)
    
    def forward(self, features):
        # print("input 2 layer FCNN features: ", features)
        features = self.fc1(features)
        # print("after 1 linear layer updated features: ", features)
        features = F.relu(features)
        # print("after ReLu updated features: ", features)
        features = self.fc2(features)
        # print("after 2 linear layers updated features: ", features)
        
        return features

# implementation of equation 5 in bondnet paper 
# https://pubs.rsc.org/en/content/articlepdf/2021/sc/d0sc05251e
class NodeFeatures(torch.nn.Module):
    # c_in1 = 24, c_out1 = 24, c_out2 = 24
    def __init__(self, c_in1, c_out1, c_out2):
        super().__init__()
        # self.fc_initial_embedding = InitialEmbedding(c_in=30, c_out=24)
        self.FCNN_one = FCNN(c_in1=c_in1, c_out1=c_out1, c_out2=c_out2)
        self.FCNN_two = FCNN(c_in1=c_in1, c_out1=c_out1, c_out2=c_out2)
        
    def forward(self, node_features, edge_index, edge_features):
        sigmoidFunction = torch.nn.Sigmoid()
        
        original_node_features = node_features.detach().clone()
        
        epsilon = 1e-7
        
        for i in range(len(node_features)):
            # DOUBLE CHECK WITH DAS
            # intermediate_node_feature = self.FCNN_one(node_features[i].T)
            intermediate_node_feature = self.FCNN_one(original_node_features[i].T)
            
            other_nodes_indices = []
            other_edges_indices = []
            
            other_edges_numerators = []
            other_edges_denominator = epsilon
            
            '''
            print("node_features[i].T: ", node_features[i].T)
            print("node_features[i].T.size: ", node_features[i].T.size())
            
            print("intermediate_node_feature: ", intermediate_node_feature)
            print("intermediate_node_feature.size: ", intermediate_node_feature.size())
            '''
            
            for j in range(len(edge_index[0])):
                if edge_index[0][j] == i:
                    other_nodes_indices.append(int(edge_index[1][j]))
                    other_edges_indices.append(j)
                if edge_index[1][j] == i:
                    other_nodes_indices.append(int(edge_index[0][j]))
                    other_edges_indices.append(j)
            
            '''
            print("current node index: ", i)
            print("other_nodes_indices: ", other_nodes_indices)
            print("other_edges_indices: ", other_edges_indices)
            '''
            
            for other_edge_index in other_edges_indices:
                other_edges_numerators.append(sigmoidFunction(edge_features[other_edge_index]))
                other_edges_denominator += sigmoidFunction(edge_features[other_edge_index])
                
            '''
            print("other_edges_numerators: ", other_edges_numerators)
            print("other_edges_denominator: ", other_edges_denominator)
            '''
            
            for other_edge_numerator, other_node_index in zip(other_edges_numerators, other_nodes_indices):
                edge_hat = other_edge_numerator/other_edges_denominator
                # DOUBLE CHECK WITH DAS
                # other_node_updated = self.FCNN_two(node_features[other_node_index].T) 
                other_node_updated = self.FCNN_two(original_node_features[other_node_index].T) 
                intermediate_node_feature += edge_hat * other_node_updated
                
                print("edge_hat: ", edge_hat)
                '''
                print("node_features[other_node_index].T: ", node_features[other_node_index].T)
                print("node_features[other_node_index].T.size: ", node_features[other_node_index].T.size())
                print("other_node_updated: ", other_node_updated)
                print("other_node_updated.size: ", other_node_updated.size())
                print("intermediate_node_feature: ", intermediate_node_feature)
                print("intermediate_node_feature.size: ", intermediate_node_feature.size())
                '''
            
            print(other_edges_numerators)
            print(other_edges_denominator)
            
            '''
            print("reLuOutput: ", F.relu(intermediate_node_feature))
            print("reLuOutput.size: ", F.relu(intermediate_node_feature).size())
            print("original_node_features[i].T", original_node_features[i].T)
            print("original_node_features[i].T.size", original_node_features[i].T.size())
            print("calculated updated node_features[i]: ", (original_node_features[i].T + F.relu(intermediate_node_feature)).T)
            print("calculated updated node_features[i].size(): ", (original_node_features[i].T + F.relu(intermediate_node_feature)).T.size())
            '''
            
            node_features[i] = (original_node_features[i].T + F.relu(intermediate_node_feature)).T
            
            '''
            print("actually updated node_features[i]: ", node_features[i])
            print("actually updated node_features[i].size(): ", node_features[i].size())
            print("********** NODE UPDATED SUCCESSFULLY ****************")
            '''
            
        return node_features
        
# implementation of equation 4 in bondnet paper
# https://pubs.rsc.org/en/content/articlepdf/2021/sc/d0sc05251e
class EdgeFeatures(torch.nn.Module):
    # c_in1 = 24, c_out1 = 24, c_out2 = 24
    def __init__(self, c_in1, c_out1, c_out2):
        super().__init__()
        # self.fc_initial_embedding = InitialEmbedding(c_in=11, c_out=24)
        self.FCNN_one = FCNN(c_in1=c_in1, c_out1=c_out1, c_out2=c_out2)
        self.FCNN_two = FCNN(c_in1=c_in1, c_out1=c_out1, c_out2=c_out2)
        
    def forward(self, node_features, edge_index, edge_features):
        original_edge_features = edge_features.detach().clone()
        
        for i in range(len(edge_index[0])):
            # summing node features involved in the given edge and transforming them
            firstNodeIndex = int(edge_index[0][i])
            secondNodeIndex = int(edge_index[1][i])
            node_features_sum = node_features[firstNodeIndex] + node_features[secondNodeIndex]
            intermediate_node_features = self.FCNN_one(node_features_sum.T)
            
            print("firstNodeIndex: ", firstNodeIndex)
            print("secondNodeIndex: ", secondNodeIndex)
            print("node_features[firstNodeIndex]: ", node_features[firstNodeIndex])
            print("node_features[secondNodeIndex]: ", node_features[secondNodeIndex])
            print("node_features_sum: ", node_features_sum)
            print("node_features_sum.size: ", node_features_sum.size())
            print("node_features_sum.T: ", node_features_sum.T)
            print("node_features_sum.T.size: ", node_features_sum.T.size())
            print("intermediate_node_features: ", intermediate_node_features)
            print("intermediate_node_features.size: ", intermediate_node_features.size())
            
            # transforming the features of the given edge 
            intermediate_edge_feature = self.FCNN_two(edge_features[i].T)
            
            print("edge_features index: ", i)
            print("edge_features: ", edge_features[i])
            print("edge_features.size: ", edge_features[i].size())
            print("edge_features.T: ", edge_features[i].T)
            print("edge_features.T.size(): ", edge_features[i].T.size())
            print("intermediate_edge_feature: ", intermediate_edge_feature)
            print("intermediate_edge_feature.size: ", intermediate_edge_feature.size())

            # merging node features with features of the given edge
            intermediate_features = F.relu(intermediate_node_features + intermediate_edge_feature)
            
            print("intermediate_features: ", intermediate_features)
            print("intermediate_features.size: ", intermediate_features.size())
            print("original_edge_features[i].T: ", original_edge_features[i].T)
            print("calculated updated edge_features[i]: ", (original_edge_features[i].T + intermediate_features).T)
            
            # updating edge features
            edge_features[i] = (original_edge_features[i].T + intermediate_features).T
            
            print("actually updated edge_features[i]: ", edge_features[i])
            print("********** EDGE UPDATED SUCCESSFULLY ****************")
            
        return edge_features

In [134]:
# UNIT TESTING EDGE UPDATE

node_features = torch.Tensor([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])
edge_index = torch.Tensor([[2, 0, 0, 1],
                       [0, 2, 1, 0]])
edge_features = torch.Tensor([[1, 0, 1],
                         [1, 0, 1],
                         [0, 1, 0],
                         [0, 1, 0]])

def linear_layer(weights, biases, features):
    # (3 x 3 x (1 x 3).T + (1 x 3).T).T returns 3 x 1
    return (np.dot(weights, features.T) + biases.T).T

def fcnn(weights, biases, features):
    features = linear_layer(weights, biases, features)
    features = F.relu(torch.Tensor(features)).numpy()
    features = linear_layer(weights, biases, features)
    return features

weights = np.array([[1., 0., 0.], 
                    [0., 1., 0.],
                    [0., 0., 1.]])

biases = np.array([0., 0., 0.])

edge_features = np.array([[1, 0, 1],
                          [1, 0, 1],
                          [0, 1, 0],
                          [0, 1, 0]])

# phi 2 test
phi_2_output = []

for edge in edge_features:
    phi_2_output.append(fcnn(weights, biases, edge))

In [135]:
# UNIT TESTING EDGE UPDATE

def linear_layer(weights, biases, features):
    # (3 x 3 x (1 x 3).T + (1 x 3).T).T returns 3 x 1
    return (np.dot(weights, features.T) + biases.T).T

def fcnn(weights, biases, features):
    features = linear_layer(weights, biases, features)
    features = F.relu(torch.Tensor(features)).numpy()
    features = linear_layer(weights, biases, features)
    return features

weights = np.array([[1., 0., 0.], 
                    [0., 1., 0.],
                    [0., 0., 1.]])

biases = np.array([0., 0., 0.])

node_features = np.array([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])

# phi 1 test
phi_1_output = []

for i in range(len(edge_index[0])):
    # summing node features involved in the given edge and transforming them
    firstNodeIndex = int(edge_index[0][i])
    secondNodeIndex = int(edge_index[1][i])
    node_features_sum = node_features[firstNodeIndex] + node_features[secondNodeIndex]
    phi_1_output.append(fcnn(weights, biases, node_features_sum))

In [136]:
# UNIT TESTING EDGE UPDATE

phi_1_np_output = np.array(phi_1_output)
phi_2_np_output = np.array(phi_2_output)

print(phi_1_np_output)
print(phi_2_np_output)

updated_edge_features = edge_features + F.relu(torch.Tensor(phi_1_np_output + phi_2_np_output)).numpy()

print(updated_edge_features)

[[1. 1. 0.]
 [1. 1. 0.]
 [0. 2. 1.]
 [0. 2. 1.]]
[[1. 0. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[3. 1. 2.]
 [3. 1. 2.]
 [0. 4. 1.]
 [0. 4. 1.]]


In [137]:
# UNIT TESTING NODE UPDATE

def linear_layer(weights, biases, features):
    # (3 x 3 x (1 x 3).T + (1 x 3).T).T returns 3 x 1
    return (np.dot(weights, features.T) + biases.T).T

def fcnn(weights, biases, features):
    features = linear_layer(weights, biases, features)
    features = F.relu(torch.Tensor(features)).numpy()
    features = linear_layer(weights, biases, features)
    return features

weights = np.array([[1., 0., 0.], 
                    [0., 1., 0.],
                    [0., 0., 1.]])

biases = np.array([0., 0., 0.])

original_node_features = np.array([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])

node_features = np.array([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])

# phi 4 test
phi_4_output = []

for node in original_node_features:
    phi_4_output.append(fcnn(weights, biases, node))

In [145]:
# UNIT TESTING NODE UPDATE

def linear_layer(weights, biases, features):
    # (3 x 3 x (1 x 3).T + (1 x 3).T).T returns 3 x 1
    return (np.dot(weights, features.T) + biases.T).T

def fcnn(weights, biases, features):
    features = linear_layer(weights, biases, features)
    features = F.relu(torch.Tensor(features)).numpy()
    features = linear_layer(weights, biases, features)
    return features

weights = np.array([[1., 0., 0.], 
                    [0., 1., 0.],
                    [0., 0., 1.]])

biases = np.array([0., 0., 0.])

node_features = np.array([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])

# phi 5 test
phi_5_output = []

sigmoidFunction = torch.nn.Sigmoid()     
epsilon = 1e-7

for i in range(len(node_features)):
    other_nodes_indices = []
    other_edges_indices = []

    other_edges_numerators = []
    other_edges_denominator = epsilon
            
    for j in range(len(edge_index[0])):
        if edge_index[0][j] == i:
            other_nodes_indices.append(int(edge_index[1][j]))
            other_edges_indices.append(j)
        if edge_index[1][j] == i:
            other_nodes_indices.append(int(edge_index[0][j]))
            other_edges_indices.append(j)
            
    for other_node_index in other_nodes_indices:
        # print("input: ", node_features[other_node_index].T)
        # print("phi 5 output: ", fcnn(weights, biases, original_node_features[other_node_index].T))
        
        original_node_features = node_features.copy()
        
        epsilon = 1e-7
        
        for i in range(len(node_features)):
            other_nodes_indices = []
            other_edges_indices = []
            
            other_edges_numerators = []
            other_edges_denominator = epsilon
            
            for j in range(len(edge_index[0])):
                if edge_index[0][j] == i:
                    other_nodes_indices.append(int(edge_index[1][j]))
                    other_edges_indices.append(j)
                if edge_index[1][j] == i:
                    other_nodes_indices.append(int(edge_index[0][j]))
                    other_edges_indices.append(j)
            
            for other_edge_index in other_edges_indices:
                sigmoidOutput = sigmoidFunction(torch.from_numpy(edge_features[other_edge_index]))
                other_edges_numerators.append(sigmoidOutput.numpy())
                sigmoidOutput = sigmoidFunction(torch.from_numpy(edge_features[other_edge_index]))
                other_edges_denominator += sigmoidOutput.numpy()

            print(other_edges_numerators)
            print(other_edges_denominator)

TypeError: expected np.ndarray (got Tensor)

In [139]:
# UNIT TESTING NODE UPDATE

phi_4_np_output = np.array(phi_4_output)

print(phi_4_np_output)

[[0. 1. 0.]
 [0. 1. 1.]
 [1. 0. 0.]]


In [140]:
# one layer test of node and edge update equations
node_features = torch.Tensor([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])
edge_index = torch.Tensor([[2, 0, 0, 1],
                       [0, 2, 1, 0]])
edge_features = torch.Tensor([[1, 0, 1],
                         [1, 0, 1],
                         [0, 1, 0],
                         [0, 1, 0]])

print("node_features: ", node_features)
print("edge_index: ", edge_index)
print("edge_features: ", edge_features)

# Node update layer
node_update_layer = NodeFeatures(c_in1=3, c_out1=3, c_out2=3)

# FCNN_one
# Linear layer 1
node_update_layer.FCNN_one.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_one.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
node_update_layer.FCNN_one.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_one.fc2.bias.data = torch.Tensor([0., 0., 0.])

# FCNN_two
# Linear layer 1
node_update_layer.FCNN_two.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_two.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
node_update_layer.FCNN_two.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_two.fc2.bias.data = torch.Tensor([0., 0., 0.])

# Edge update layer
edge_update_layer = EdgeFeatures(c_in1=3, c_out1=3, c_out2=3)

# FCNN_one
# Linear layer 1
edge_update_layer.FCNN_one.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_one.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Lienar layer 2
edge_update_layer.FCNN_one.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_one.fc2.bias.data = torch.Tensor([0., 0., 0.])

# FCNN_two
# Linear layer 1
edge_update_layer.FCNN_two.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_two.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
edge_update_layer.FCNN_two.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_two.fc2.bias.data = torch.Tensor([0., 0., 0.])

with torch.no_grad():
    node_out_features = node_update_layer(node_features, edge_index, edge_features)
    # edge_out_features = edge_update_layer(node_features, edge_index, edge_features)

print("after layer propogation 1")
print("node_out_features: ", node_out_features)
# print("edge_out_features: ", edge_out_features)

node_features:  tensor([[0., 1., 0.],
        [0., 1., 1.],
        [1., 0., 0.]])
edge_index:  tensor([[2., 0., 0., 1.],
        [0., 2., 1., 0.]])
edge_features:  tensor([[1., 0., 1.],
        [1., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]])
edge_hat:  tensor([0.2969, 0.2031, 0.2969])
edge_hat:  tensor([0.2969, 0.2031, 0.2969])
edge_hat:  tensor([0.2031, 0.2969, 0.2031])
edge_hat:  tensor([0.2031, 0.2969, 0.2031])
[tensor([0.7311, 0.5000, 0.7311]), tensor([0.7311, 0.5000, 0.7311]), tensor([0.5000, 0.7311, 0.5000]), tensor([0.5000, 0.7311, 0.5000])]
tensor([2.4621, 2.4621, 2.4621])
edge_hat:  tensor([0.5000, 0.5000, 0.5000])
edge_hat:  tensor([0.5000, 0.5000, 0.5000])
[tensor([0.5000, 0.7311, 0.5000]), tensor([0.5000, 0.7311, 0.5000])]
tensor([1.0000, 1.4621, 1.0000])
edge_hat:  tensor([0.5000, 0.5000, 0.5000])
edge_hat:  tensor([0.5000, 0.5000, 0.5000])
[tensor([0.7311, 0.5000, 0.7311]), tensor([0.7311, 0.5000, 0.7311])]
tensor([1.4621, 1.0000, 1.4621])
after layer propogat

In [80]:
# two layer test of node and edge update equations
node_features = torch.Tensor([[0, 1, 0],
                          [0, 1, 1],
                          [1, 0, 0]])
edge_index = torch.Tensor([[2, 0, 0, 1],
                       [0, 2, 1, 0]])
edge_features = torch.Tensor([[1, 0, 1],
                         [1, 0, 1],
                         [0, 1, 0],
                         [0, 1, 0]])

print("node_features: ", node_features)
print("edge_index: ", edge_index)
print("edge_features: ", edge_features)

# Node update layer
node_update_layer = NodeFeatures(c_in1=3, c_out1=3, c_out2=3)

# FCNN_one
# Linear layer 1
node_update_layer.FCNN_one.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_one.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
node_update_layer.FCNN_one.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_one.fc2.bias.data = torch.Tensor([0., 0., 0.])

# FCNN_two
# Linear layer 1
node_update_layer.FCNN_two.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_two.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
node_update_layer.FCNN_two.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
node_update_layer.FCNN_two.fc2.bias.data = torch.Tensor([0., 0., 0.])

# Edge update layer
edge_update_layer = EdgeFeatures(c_in1=3, c_out1=3, c_out2=3)

# FCNN_one
# Linear layer 1
edge_update_layer.FCNN_one.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_one.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Lienar layer 2
edge_update_layer.FCNN_one.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_one.fc2.bias.data = torch.Tensor([0., 0., 0.])

# FCNN_two
# Linear layer 1
edge_update_layer.FCNN_two.fc1.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_two.fc1.bias.data = torch.Tensor([0., 0., 0.])

# Linear layer 2
edge_update_layer.FCNN_two.fc2.weight.data = torch.Tensor([[1., 0., 0.], 
                                             [0., 1., 0.],
                                             [0., 0., 1.]])
edge_update_layer.FCNN_two.fc2.bias.data = torch.Tensor([0., 0., 0.])

for i in range(2):
    with torch.no_grad():
        # node_out_features = node_update_layer(node_features, edge_index, edge_features)
        edge_out_features = edge_update_layer(node_features, edge_index, edge_features)
        
    print("after layer propogation ", i + 1)
    # print("node_out_features: ", node_out_features)
    print("edge_out_features: ", edge_out_features)

node_features:  tensor([[0., 1., 0.],
        [0., 1., 1.],
        [1., 0., 0.]])
edge_index:  tensor([[2., 0., 0., 1.],
        [0., 2., 1., 0.]])
edge_features:  tensor([[1., 0., 1.],
        [1., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]])
input 2 layer FCNN features:  tensor([1., 1., 0.])
after 1 linear layer updated features:  tensor([1., 1., 0.])
after ReLu updated features:  tensor([1., 1., 0.])
after 2 linear layers updated features:  tensor([1., 1., 0.])
firstNodeIndex:  2
secondNodeIndex:  0
node_features[firstNodeIndex]:  tensor([1., 0., 0.])
node_features[secondNodeIndex]:  tensor([0., 1., 0.])
node_features_sum:  tensor([1., 1., 0.])
node_features_sum.size:  torch.Size([3])
node_features_sum.T:  tensor([1., 1., 0.])
node_features_sum.T.size:  torch.Size([3])
intermediate_node_features:  tensor([1., 1., 0.])
intermediate_node_features.size:  torch.Size([3])
input 2 layer FCNN features:  tensor([1., 0., 1.])
after 1 linear layer updated features:  tensor([1., 0., 

In [81]:
print("node update layers: ")
print(node_update_layer.FCNN_one.fc1.weight)
print(node_update_layer.FCNN_one.fc1.bias)
print(node_update_layer.FCNN_two.fc2.weight)
print(node_update_layer.FCNN_two.fc2.bias)

print("\nedge update layers: ")
print(edge_update_layer.FCNN_one.fc1.weight)
print(edge_update_layer.FCNN_one.fc1.bias)
print(edge_update_layer.FCNN_two.fc2.weight)
print(edge_update_layer.FCNN_two.fc2.bias)

node update layers: 
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)

edge update layers: 
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)
